In [44]:
import pandas as pd
import numpy as np
import os
from pandas.tseries.offsets import BDay
import pickle
import fnmatch
from scipy import stats
from dateutil.parser import parse
from pylab import *
import statsmodels as sts
import datetime as dt
import glob
from scipy.spatial.distance import pdist, squareform
def listdirs(folder): #return only directories from a master folder
    return [d for d in os.listdir(folder) if os.path.isdir(os.path.join(folder, d))]

def find_path(basepath, stringname):
    for fname in os.listdir(basepath):
        path = os.path.join(basepath, fname)
        if os.path.isdir(path):
            if stringname in fname:
                return path
def get_ticket(rx_path):
    ticker=rx_path.split(".")[0].split("/")[-1].split("-")[0]
    return ticker

def ticker_trades_dir(ticker):
    dest=os.path.join(os.getenv('FINANCE_DATA'), "_".join((ticker,'trades')))
    if not os.path.isdir(dest):
        os.makedirs(dest)
def quotes_trades_dir(ticker):
    dest=os.path.join(os.getenv('FINANCE_DATA'), "_".join((ticker,'quotes')))
    if not os.path.isdir(dest):
        os.makedirs(dest)
def agg_on_trd_time(gr):
    """
    Utility func to aggregate trades on timestamp. All trades with equal time stamp
    will collapse to one row and the traded price will be the volume weighted traded
    price.
    """
    vTrdPrice = np.sum(gr['TradedPrice'] * gr['Volume'])/np.sum(gr['Volume'])
    volume = np.sum(gr['Volume'])

    return pd.Series({'Volume': volume,
                      'TradedPrice': vTrdPrice})

In [2]:
data_dir = os.getenv('FINANCE_DATA') #main directory
data_only_drive= '/mnt/usb-Seagate_Expansion_Desk_NA8XEHR6-0:0-part2'
bmrg_symbols=[s for s in os.listdir(data_dir) if s.endswith('20181028') or s.endswith('20181027')]


bmrg_symbols_destinations=[s for s in os.listdir(data_dir) if s.endswith('trades') or s.endswith('quotes')]
bmrg_tickers=[bmrg_symbols_destinations[idx].split('_t')[0] or \
              bmrg_symbols_destinations[idx].split('_q')[0] for idx,_ in enumerate(bmrg_symbols_destinations)]

In [3]:
bmrg_folders=[s for s in os.listdir(data_dir) if s.endswith('Comdty')]
bmrg_trades=[s for s in os.listdir(data_dir) if s.endswith('y_trades')]
bmrg_quotes=[s for s in os.listdir(data_dir) if s.endswith('y_quotes')]
bmrg_tickers=[bmrg_trades[idx].split('_t')[0] for idx,_ in enumerate(bmrg_trades)]

In [10]:
#need these columns 'TradedTime', 'TradedPrice', 'ReturnTradedPrice', 'Volume','Duration'

In [12]:
symbol='US1_Comdty'
trades_files_loc = os.path.join(data_dir,symbol)
trades_files=os.listdir(trades_files_loc)
file_idx=3
trades_df = pd.read_csv(os.path.join(trades_files_loc, trades_files[file_idx]), index_col=False)
trades_df['SquaredReturns']=trades_df['ReturnTradedPrice']**2
trades_df=trades_df.dropna()
trades_df['TradedDateTime']=pd.to_datetime(trades_df['TradedTime'])

In [20]:
type(trades_df['TradedDateTime'])

pandas.core.series.Series

In [ ]:
trades_df.fillna(value=0, inplace=True)
trades_df.columns.values


In [67]:
test=trades_df['TradedTime'][2]


xrange(10)

In [72]:
# datetime.datetime.strptime(test,'%Y-%m-%d %H:%M:%S')
#datetime.strptime(war_start, '%Y-%m-%d')
parse(test).date()
trades_df['TradedTime']=trades_df['TradedTime'].apply(parse)

In [86]:
# trades_df['TradedTime'].datetime.time()
trades_df['TradedTime'] = [datetime.datetime.time(d) for d in trades_df['TradedTime']]

In [89]:
trades_df= trades_df.drop(columns=['TradedDateTime'])

In [91]:
trades_df.columns.values

array(['TradedTime', 'TradedPrice', 'ReturnTradedPrice', 'Volume',
       'Duration', 'SquaredReturns'], dtype=object)

In [ ]:
fig, ax = subplots()
nonz=trades_df[trades_df['ReturnTradedPrice']!=0]
zero=trades_df[trades_df['ReturnTradedPrice']==0]
ax.scatter(nonz['Volume'], nonz['Duration'], cmap='gray')
# ax.set_aspect(1)
# xlim(np.min(z['ReturnTradedPrice']), np.max(z['ReturnTradedPrice']))
# ylim(np.max(z['Duration']), np.max(z['Duration']))
xlabel('Volume')
ylabel('Duration')
# title('Volume') ;
ax.scatter(zero['Volume'], zero['Duration'], cmap='gray')


In [ ]:
fig, ax = subplots()
ax.scatter(zero['Volume'], zero['Duration'], cmap='gray')

In [ ]:
trades_df['simple_rtn'] = 100 * trades_df['ReturnTradedPrice']
trades_df['log_rtn'] = 100 * (np.log(trades_df['TradedPrice']) - np.log(trades_df['TradedPrice'].shift(1)))

# dropping NA's in the first row
trades_df.dropna(how = 'any', inplace = True)

trades_df.columns.values

In [ ]:
df_nonzero = trades_df[trades_df['ReturnTradedPrice']!=0]
df_zero= trades_df[trades_df['ReturnTradedPrice']==0]

In [ ]:
sns.pairplot(df_nonzero, hue="Volume")

In [ ]:
x_nz = df_nonzero['ReturnTradedPrice']
dur_nz= df_nonzero['Duration']
vol_nz = df_nonzero['Volume']
sns.scatterplot(x=x_nz, y=dur_nz, hue=vol_nz,sizes=(1, 8),palette="ch:r=-.3,d=.7_r", linewidth=0)
# Plot a historgram and kernel density estimate


In [ ]:
sns.distplot(dur_nz, color="m")

In [ ]:
import scipy.stats as scs

# Plotting the distribution of the returns ----
ax = sns.distplot(x_nz, kde = True, norm_hist=True)                                    

xx = np.arange(min(x_nz), max(x_nz), 0.01)                                                   
yy = scs.norm.pdf(xx)                                                         
ax.plot(xx, yy, 'r', lw=2)
_=ax.set_title('Distribution of returns')

In [ ]:
import statsmodels.formula.api as smf
import pandas as pd
import matplotlib.pyplot as plt
from pandas.plotting import autocorrelation_plot

In [ ]:
ticker_data =x_nz
ticker_data_len = len(ticker_data)
sns.distplot(x_nz)